In [1]:
import sys
sys.path.insert(1, "./src")

In [2]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from datasets.ner_dataset import get_ner_dataset, NerDataset

In [4]:
def plot_class_distribution(data: NerDataset, title: str) -> pd.DataFrame:
    token_list = []
    for i in range(len(data)):
        token_list.extend(list(map(lambda x: (x[0].item(), data.index2label[x[1].item()]), 
                                   zip(*data[i][1:]))))
    token_list = list(filter(lambda x: x[   1].strip() != '[PAD]', token_list))
    print(f"The number of all tokens: {len(token_list)}")
    df = pd.DataFrame(token_list, columns=['token','label'])
    print(df['label'].value_counts(normalize = False))
    distr = df['label'].loc[df['label'] != 'O'].value_counts(normalize = True)
    plt.figure(figsize=(15, 2.5))
    with sns.plotting_context("talk", font_scale=0.5), sns.axes_style('whitegrid'), sns.mpl_palette("GnBu_d", n_colors=len(distr)):
        sns.barplot(x=distr.index, y=distr.values).set_title(title)
    return df

In [5]:
def plot_len_distribution(data: NerDataset):
    len_list = [len(data[i][1]) for i in range(len(data))]
    print(f"The number of records: {len(len_list)}")
    df = pd.DataFrame({'length': len_list})
    print(f"The mean length: {df['length'].mean()}")
    plt.figure(figsize=(16, 8))
    with sns.plotting_context("talk", font_scale=0.7), sns.axes_style('whitegrid'), sns.mpl_palette("GnBu_d"):
        sns.histplot(df['length'], kde=False, bins=50).set_title("Распределение количества токенов в записях")

### Этап 5: Дополнительное исследование данных

Анализ данных будет проходить на основе токенизатора основной модели "bert-large-uncased".

#### i2b2 2014

In [6]:
data_config = yaml.load(open("configs/i2b2-2014_data_config.yaml", 'r'), Loader=yaml.Loader)
data_config['eq_max_padding'] = False
data_config['max_length'] = None
train_dataset = get_ner_dataset(path_to_folder=data_config["train_data_path"], device="cpu", **data_config)
val_dataset = get_ner_dataset(path_to_folder=data_config["validate_data_path"], device="cpu", **data_config)

In [7]:
train_df = plot_class_distribution(train_dataset, 'Тренировочная выборка')

In [8]:
val_df = plot_class_distribution(val_dataset, 'Тестовая выборка')

Заметим, что около $90\%$  токенов являются 'OTHER' и распозоваемые сущности занимают менее 10% всего текста, т.к одна четверть токенов являются техническими. Распознаваемые классы распределены неравномерно. Среди них самым популярным является класс 'DATE', оставшиеся классы встречаются в два раза реже. А самым редким классом, который определён только у 830 токенов, является 'PROFESSION', что приводит к относительно низкому качеству его распозавания всеми моделями.  
Распределение классов в тренировочной и валидационной выборке совпадает. При этом обучающая выборка лишь в полтора раза больше тестовой.

In [9]:
plot_len_distribution(train_dataset)

In [10]:
plot_len_distribution(val_dataset)

Распределение длин одинаково для тренировочной и валидационной выборок и схоже с колоколом распределения Пуассона.  
Примечательно, что средняя длина в обоих наборах данных почти совпадает.

#### i2b2 2006

In [11]:
data_config = yaml.load(open("configs/i2b2-2006_data_config.yaml", 'r'), Loader=yaml.Loader)
data_config['eq_max_padding'] = False
data_config['max_length'] = None
train_dataset = get_ner_dataset(path_to_folder=data_config["train_data_path"], device="cpu", **data_config)
val_dataset = get_ner_dataset(path_to_folder=data_config["validate_data_path"], device="cpu", **data_config)

In [12]:
train_df = plot_class_distribution(train_dataset, 'Тренировочная выборка')

In [13]:
val_df = plot_class_distribution(val_dataset, 'Тестовая выборка')

Заметим, что около $90\%$  токенов являются 'OTHER' и распозоваемые сущности занимают менее 10% всего текста, т.к одна пятая токенов являются техническими. Распознаваемые классы распределены более равномерно, чем в наборе данных за 2014 год, однако в нём отсутствует класс 'PROFESSION' и почти не представлен класс 'AGE' (51 токен). Среди присутсвующих самым популярным является класс 'NAME' --- $25\%$ токенов сущностей, оставшиеся классы занимают $<21\%$. А самым редким классом, который определён только у 714 токенов, является 'CONTACT', занимая лишь $~2\%$ токенов.  
Распределение классов в тренировочной и валидационной выборке совпадает. При этом обучающая выборка примерно в три раза больше тестовой.

In [14]:
plot_len_distribution(train_dataset)

In [15]:
plot_len_distribution(val_dataset)

Распределение длин текстов в тренировочной и тестовой выборках несколько отличается: если в первой наблюдаются очертания колокола Пуассоновского распределения, то во второй распределение более пологое и ближе к равномерному.  
Стоит отметить, что средняя длина валидационных текстов на 200 токенов больше аналогичной у тренировочных.